<h1><b><center>How to use this file with PyRat?</center></b></h1>

Google Colab provides an efficient environment for writing codes collaboratively with your group. For us, teachers, it allows to come and see your advancement from time to time, and help you solve some bugs if needed.

The PyRat software is a complex environment that takes as an input an AI file (as this file). It requires some resources as well as a few Python libraries, so we have installed it on a virtual machine for you.

PyRat is a local program, and Google Colab is a distant tool. Therefore, we need to indicate the PyRat software where to get your code! In order to submit your program to PyRat, you should follow the following steps:

1.   In this notebook, click on *Share* (top right corner of the navigator). Then, change sharing method to *Anyone with the link*, and copy the sharing link;
2.   On the machine where the PyRat software is installed, start a terminal and navigate to your PyRat directory;
3.   Run the command `python ./pyrat.py --rat "<link>" <options>`, where `<link>` is the share link copied in step 1. (put it between quotes), and `<options>` are other PyRat options you may need.

<h1><b><center>Pre-defined constants</center></b></h1>

A PyRat program should -- at each turn -- decide in which direction to move. This is done in the `turn` function later in this document, which should return one of the following constants:

In [ ]:
MOVE_DOWN = 'D'
MOVE_LEFT = 'L'
MOVE_RIGHT = 'R'
MOVE_UP = 'U'

<h1><b><center>Your coding area</center></b></h1>

Please put your functions, imports, constants, etc. between this text and the PyRat functions below. You can add as many code cells as you want, we just ask that you keep things organized (one function per cell, commented, etc.), so that it is easier for the teachers to help you debug your code!

In [ ]:
moves=[]
import heapq

In [ ]:
def move_from_locations(source_location, target_location):

    # Find difference between current and target location
    difference = (target_location[0] - source_location[0], target_location[1] - source_location[1])

    # Return the corresponding move
    if difference == (0, -1) :
        return MOVE_DOWN
    elif difference == (0, 1) :
        return MOVE_UP
    elif difference == (1, 0) :
        return MOVE_RIGHT
    elif difference == (-1, 0) :
        return MOVE_LEFT
    else :
        raise Exception("Impossible move")

In [ ]:
def moves_from_locations(locations):

    # Transform a series of locations into corresponding moves to realize it
    moves = []
    for i in range(len(locations) - 1) :
        moves.append(move_from_locations(locations[i], locations[i+1]))
    return moves

In [ ]:
def find_route(routing_table, source_location, target_location):

    # Return a sequence of locations from source to target using provided routing table
    route = [target_location]
    while route[0] != source_location :
        route.insert(0, routing_table[route[0]])
    return route

In [ ]:
def new_min_heap() :

    # Create an empty min_heap
    return []

In [ ]:
def add_or_replace(min_heap, value, vertex, parent):
  # entrée: min_heap (list), vertex(value), parent(value)
  # sortie: min_heap dans laquelle on a ajouté ou modifié la distance associé au vertex

    s=False
    for i in range(len(min_heap)):
        if min_heap[i][1]== vertex:# on verifie si le vertex existe déjà dans la min_heap
            b= min_heap[i][0]
            s=True
            if b> value:# on compare les distances
                del min_heap[i] # on ne peut pas modifier les tuples il faut donc supprimer l'ancien et ajouter le nouveau couple
                heapq.heappush(min_heap, (value, vertex, parent))
    if s==False:#le cas où le vertex n'existe pas deja dans la min_heap
        heapq.heappush(min_heap, (value, vertex, parent))
    return min_heap

In [ ]:
def fast_dijkstra(start_vertex, target_vertices, maze_map) :
    #entrée: current_vertex("start_vertex"), list des sommets visés, graph
    #sortie: liste des explored_vertices, la routing_table et le dernier vertex visité
    #la différence avec un dijkstra classique est que fast_dijkstra arrète l'exploration des sommets dès qu'il atteint un fromage qui sera, dès lors, le fromage le plus proche géographiquement

    # Initialize structure with initial vertex, at distance 0, with no predecessor
    min_heap= new_min_heap()
    add_or_replace(min_heap, 0, start_vertex, None)

    # Initialize routing (main difference with course is we also store the length of paths stored with explored vertices)
    explored_vertices = {}
    routing_table = {}
    # Loop until all vertices have been explored
    while len(min_heap) > 0 :
        # Pop next vetex to visit
          distance, vertex, parent = heapq.heappop(min_heap)
          if vertex not in explored_vertices :

            # Store route to it
            explored_vertices[vertex] = distance
            routing_table[vertex]= parent

            # Add its its neighbors to the structure for later consideration
            for neighbor in list(maze_map[vertex].keys()) :
                if neighbor not in explored_vertices :
                    distance_to_neighbor = distance + maze_map[vertex][neighbor]
                    add_or_replace(min_heap, distance_to_neighbor, neighbor, vertex)
          for i in target_vertices:
            # si on a déjà atteint un fromage alors on arrète la boucle et on renvoie les données de sortie
            if i in explored_vertices:
               return explored_vertices, routing_table, vertex


    # Return shortest paths and routing table
    return explored_vertices, routing_table, vertex

In [ ]:
def remove_list(list, element):
  # entrée : liste, élément
  # sortie : aucune
  # enlever un élément d'une liste qui ne contient pas de doublon et dont on sait qu'elle contient cet élément
  i=0
  while list[i]!=element:
      i=i+1
  del list[i]

In [ ]:
def go_backwards(exp_player,exp_opponent,cheese_player,cheese_opponent):
  # si l'adversaire est plus proche de notre fromage le plus proche, on change de cible par précaution
  # entrée : les dictionnaires des sommets à visiter pour chacun des joueurs pour aller au fromage le plus proche (obtenus avec fast_dijkstra), la position des fromages potentiellement visés par chacun d'eux
  # sortie : booléen indiquant si on abandonne le fromage visé ou pas
  backwards=False
  if cheese_opponent==cheese_player: # on vérifie si le fromage potenitellement ciblé par l'adversaire est le nôtre
    distance_opponent=exp_opponent[cheese_opponent]
    distance_player=exp_player[cheese_player]
    if distance_opponent<=distance_player:
      backwards=True # si l'adversaire est plus proche que nous, on change de fromage, donc il faut que le booléen soit "True"
  return backwards

In [ ]:
def sabotage(exp_player,exp_opponent,cheese_opponent):
  # on pique le fromage de l'adversaire s'il est plus proche de nous que de lui, même si ce n'est ps le fromage qui minimise la distance
  # entrées : les dictionnaires des sommets à visiter pour chacun des joueurs pour aller au fromage le plus proche de l'adversaire (obtenu avec le fast_dijkstra), position du fromage le plus proche de l'adversaire
  # sortie : booléen indiquant si on pique le fromage ou pas
  sabotage=False
  distance_opponent=exp_opponent[cheese_opponent]
  distance_player=exp_player[cheese_opponent]
  if distance_player<distance_opponent:
    sabotage=True # si on est plus près du fromage le plus proche de l'adversaire, on lui prend donc on initilaise le booléen en "True"
  return sabotage

In [ ]:
def strategic_improved_greedy(maze_map, player_location, opponent_location, pieces_of_cheese):
  # entrée: graph, position du joueur et de l'adversaire, et les postions des fromages (liste)
  # sortie: liste des mouvements à suivre pour atteindre le fromage choisi
  # on commence par fast_dijkstra, pour les 2 joueurs, car la stratégie repose aussi sur la position actuelle de l'adversaire
  exp_player,rout_player,cheese_player=fast_dijkstra(player_location,pieces_of_cheese,maze_map)
  exp_opponent,rout_opponent,cheese_opponent=fast_dijkstra(opponent_location,pieces_of_cheese,maze_map)
  if go_backwards(exp_player,exp_opponent,cheese_player,cheese_opponent)==True: # on commence par regarder s'il serait mieux de changer de fromage par précaution
    remove_list(pieces_of_cheese,cheese_player) # on retire le fromage visé actuellement de la liste des fromages à visiter
    exp_player,rout_player,cheese_player=fast_dijkstra(player_location,pieces_of_cheese,maze_map) # on exécute à nouveau fast_dijkstra avec la liste de fromages qui n'inclut pas le fromage "problématique"
    route=find_route(rout_player,player_location,cheese_player)
    return moves_from_locations(route)
  exp_player_sabotage,rout_player_sabotage,cheese_player_sabotage=fast_dijkstra(player_location,[cheese_opponent],maze_map) # sinon, on réutilise fast_dijkstra pour voir s'il serait intéressant de prendre le fromage le plus proche de l'adversaire ; la liste des cibles se réduit à cheese_opponent d'après la construcution de fast_dijkstra
  if sabotage(exp_player_sabotage,exp_opponent,cheese_opponent)==True: # on regarde si on est plus proche du fromage le plus proche de l'adversaire
    route=find_route(rout_player_sabotage,player_location,cheese_opponent)
    return moves_from_locations(route)
  route=find_route(rout_player,player_location,cheese_player) # sinon, aucune des deux stratégies n'est déployée donc on va simplement au fromage le plus proche de nous
  return moves_from_locations(route)

<h1><b><center>PyRat functions</center></b></h1>

The `preprocessing` function is called at the very start of a game. It is attributed a longer time compared to `turn`, which allows you to perform intensive computations. If you store the results of these computations into **global** variables, you will be able to reuse them in the `turn` function.

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze where the game takes place. This structure associates each cell with the dictionry of its neighbors. In that dictionary of neighbors, keys are cell coordinates, and associated values the number of moves required to reach that neighbor. As an example, `list(maze_map[(0, 0)].keys())` returns the list of accessible cells from `(0, 0)`. Then, if for example `(0, 1)` belongs to that list, one can access the number of moves to go from `(0, 0)` to `(0, 1)` by the code `maze_map[(0, 0)][0, 1)]`.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The initial location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The initial location of your opponent's character in the maze.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The initial location of all pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take for preprocessing before the game starts checking for moves.

*Output:*
*   This function does not output anything.

In [ ]:
def preprocessing (maze_map, maze_width, maze_height, player_location, opponent_location, pieces_of_cheese, time_allowed) :

    # Example prints that appear in the shell only at the beginning of the game
    # Remove them when you write your own program
    print("maze_map", type(maze_map), maze_map)
    print("maze_width", type(maze_width), maze_width)
    print("maze_height", type(maze_height), maze_height)
    print("player_location", type(player_location), player_location)
    print("opponent_location", type(opponent_location), opponent_location)
    print("pieces_of_cheese", type(pieces_of_cheese), pieces_of_cheese)
    print("time_allowed", type(time_allowed), time_allowed)

The `turn` function is called each time the game is waiting
for the player to make a decision (*i.e.*, to return a move among those defined above).

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze. It is the same as in the `preprocessing` function, just given here again for convenience.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The current location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The current location of your opponent's character in the maze.
*   `player_score` -- **float** -- Your current score.
*   `opponent_score` -- **float** -- The opponent's current score.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The location of remaining pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take to return a move to apply before another time starts automatically.

*Output:*
*   A chosen move among `MOVE_UP`, `MOVE_DOWN`, `MOVE_LEFT` or `MOVE_RIGHT`.

In [ ]:
def turn (maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed) :
    global moves
    # contrairement au greedy, on fait appel à la fonction improved_greedy à chaque turn. Ainsi, la liste des fromages sera modifiée si l'adversaire a deja pris un fromage vers lequel on se dirige
    #On prend en compte la modification des fromages restant dû fait d'un adversaire présent dans la maze
    moves= strategic_improved_greedy(maze_map, player_location,opponent_location, pieces_of_cheese)
    return moves.pop(0)